# Seq2Seq para tradução inglês - Português

Modelo seq2seq simples para tradução de texto de inglês para português com leitura character por character.

In [1]:
from keras.models import Model
from keras.layers import Input, CuDNNLSTM, Dense, LSTM
import numpy as np

latent_dim = 264  #Tamanho do estado h
num_samples = 20000  # Numero de amostras a serem usadas

#Arquivo com pares inglês-português
data_path = 'por.txt'

c:\users\vinic\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

lines = open(data_path, encoding="utf8").read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # Usaremos um \t para indicar o inicio de uma frase e \n para indicar o fim
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Número de amostras:', len(input_texts))
print('Número de tokens unicos de input:', num_encoder_tokens)
print('Number de tokens unicos de output:', num_decoder_tokens)
print('Tamanho máximo de input:', max_encoder_seq_length)
print('Tamanho máximo de output:', max_decoder_seq_length)

#input_texts = input_texts[:5000]
#target_texts = target_texts[:5000]

#cria dicionário de tokens
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

print()
print('input_char: ', input_characters)
print()
print('input_token_index: ', input_token_index)


Número de amostras: 20000
Número de tokens unicos de input: 72
Number de tokens unicos de output: 89
Tamanho máximo de input: 19
Tamanho máximo de output: 59

input_char:  [' ', '!', '$', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é', '’']

input_token_index:  {'4': 11, 'k': 54, '0': 7, 't': 63, '5': 12, '-': 5, 'e': 48, 'v': 65, 'd': 47, 'o': 58, 'A': 19, 'p': 59, 'G': 25, 'j': 53, '1': 8, '6': 13, '$': 2, ':': 17, '3': 10, 'F': 24, '9': 16, 'l': 55, 'O': 33, 'y': 68, 'T': 38, 'm': 56, 'w': 66, 'I': 27, 'a': 44, 'f': 49, 'x': 67, 'q': 60, 'z': 69, 'M': 31, 'D': 22, 'S': 37, 'h': 51, 'R': 36, '.': 6, ' ': 0, 'u': 64, '!': 1, 'U': 39, '’': 71, 'g': 50, '8': 15, 'n': 57, 'K': 29, 's': 62, 'V': 40, 'B

In [5]:
#cria arrays de 0 para fazer o padding
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

#preenche os arrays com os caracteres
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data é a saída esperada do decoder, então deve estar um tempo a frente e não incluir o \t
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

#one hot encoding
print(input_texts[0])
for i in encoder_input_data[0]:
    print(i)

Go.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

## Seq2Seq

Aqui criamos um modelo Seq2Seq simples com uma camada de LSTM como encoder e outra como decoder.

O encoder retorna seus estados que serão a inicialização do decoder



In [4]:
"""
Teacher Forcing:

Suponha a frase inicial:
Mary had a little lamb whose fleece was white as snow

A mesma frase com os marcadores de inicio e fim:
[START] Mary had a little lamb whose fleece was white as snow [END]

Treinamento:

X                        y
--------------------------
[START],                 ?
[START], Mary,           ?
[START], Mary, had,      ?
[START], Mary, had, a,   ?
...

a diferença para a inferencia é que no treinamento durante a geração da saida, 
o decoder é realimentado com a saida esperada e não a saida gerada.

"""
print()

In [5]:
from keras.models import Sequential, Model


encoder_inputs = Input(shape=(None, num_encoder_tokens))

#Encoder
encoder = LSTM(latent_dim, return_state=True)

#pega saida do encoder
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c] #mantem apenas os estados internos


decoder_inputs = Input(shape=(None, num_decoder_tokens))

#Decoder
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

#pega saída do decoder
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states) #inicializa os estados interno do decoder com os do encoder

#camada Dense com softmax do tamanho do vocabulario para definir qual o caracter de saída
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

#O modelo para o treino tem como entrada o input a ser codificado e o input do Force Teaching do decoder
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
#word-level model with integer sequences
# anteriormente deve-se separar por palavras e nao caracteres

from keras.models import Sequential, Model

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
x = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
x, state_h, state_c = LSTM(latent_dim,
                           return_state=True)(x)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
x = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
x = LSTM(latent_dim, return_sequences=True)(x, initial_state=encoder_states)
decoder_outputs = Dense(num_decoder_tokens, activation='softmax')(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [6]:
model.summary()

#model.load_weights('seq2seq_weights.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 72)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 89)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 264), (None, 355872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 264),  373824      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

Uma das formas mais elegantes de salvar os pesos e o modelo é via callbacks:

In [7]:
from keras.callbacks import ModelCheckpoint

# checkpoint
#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.h5"
filepath = "seq2seq_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [8]:
batch_size = 64
epochs = 100

# Executa o treino
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs, shuffle=False, #Sem shuffle para não misturar frases de tamanhos diferentes
          validation_split=0.1, verbose = 1, callbacks=callbacks_list)

Train on 18000 samples, validate on 2000 samples
Epoch 1/100


InternalError: Blas GEMM launch failed : a.shape=(64, 72), b.shape=(72, 264), m=64, n=264, k=72
	 [[Node: lstm_1/while/MatMul_3 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](lstm_1/while/TensorArrayReadV3, lstm_1/while/MatMul_3/Enter)]]
	 [[Node: loss/mul/_93 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3107_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'lstm_1/while/MatMul_3', defined at:
  File "c:\users\vinic\appdata\local\programs\python\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\vinic\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\vinic\appdata\local\programs\python\python35

In [ ]:
#uma forma mais simples de salvar os pesos é usando a função save_weights, que salvará o ultimo estado da rede
#model.save_weights('seq2seq_weights.h5')
#ou salva o modelo todo utilizando save
model.save('s2s_model.h5')

In [ ]:
#Pesos do treino já executado
model.load_weights('seq2seq_weights.h5')
#carrega tudo: peso e modelo
#mode.load('s2s_model.h5')

A principal diferença entre o modelo de testes e o de treinamento é que no treino utilizamos o force teaching e no teste iremos usar a real saída do decoder como input

In [ ]:
#Modelo do Encoder
encoder_model = Model(encoder_inputs, encoder_states)

#Inputs para o decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

#Reutiliza a LSTM treinada anteriormente com entradas novas
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

#Agora nosso modelo possui como entrada os estados que irão ser gerados pelo encoder
#e o output do decoder gerado pelos tempos anteriores

#Como saída temos também os estados para garantir que no próximo tempo o Decoder continue de onde parou
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
#Cria dicionário de tokens inverso para decodificar a saída
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

A função abaixo irá fazer a geração de saída do decoder, utilizando uma saída por vez e realimentando o Decoder

In [ ]:
def decode_sequence(input_seq):
    # Aplica as entradas no encoder para pegar seus estados (sua memória)
    states_value = encoder_model.predict(input_seq)

    # Gera um vetor de entrada nulo para o decoder
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # coloca o \t na primeira posição do vetor indicando para o decoder que deve iniciar a gerar texto
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Faz o loop gerando saídas do Decoder
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition: # gere novos tokens enquanto nao encontrar uma condição de parada 
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        # Pega o token de saída
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # Traduz para texto
        sampled_char = reverse_target_char_index[sampled_token_index]
        # Adiciona à string de resposta
        decoded_sentence += sampled_char

        # Se a saída for \n ou tiver atingido o tamanho máximo, para o loop
        if (sampled_char == '\n' or
               len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Recria o input do decoder com a última saída
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [ ]:
for seq_index in range(300,400):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

In [ ]:
def teste(teste):
    texto_seq = np.zeros((1,max_encoder_seq_length,num_encoder_tokens))

    for t, char in enumerate(teste):
        texto_seq[0, t, input_token_index[char]] = 1.
    
    decoded_sentence = decode_sequence(texto_seq)
    print('-')
    print('Input:', teste)
    print('Output:', decoded_sentence)

In [ ]:
teste('I want to eat an apple')